<a href="https://colab.research.google.com/github/AiNguyen2014/MachineLearningProject/blob/main/Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lưu ý

# Mình sẽ đánh giá essemble của train bằng silhoutte Và đánh giá external giữa independent với actual. Internal chỉ dùng để đánh giá train thôi

In [5]:
from google.colab import drive
import os

drive.mount('/content/drive')

TARGET_FOLDER = "Project Machine Learning"
BASE_PATH = None

for root, dirs, files in os.walk("/content/drive/MyDrive"):
    if TARGET_FOLDER in dirs:
        BASE_PATH = os.path.join(root, TARGET_FOLDER)
        break

if BASE_PATH is None:
    raise FileNotFoundError(" Không tìm thấy thư mục Project Machine Learning")

print(" Dùng thư mục:", BASE_PATH)
print(" File trong thư mục:", os.listdir(BASE_PATH))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 Dùng thư mục: /content/drive/MyDrive/Project Machine Learning
 File trong thư mục: ['actual.csv', 'data_set_ALL_AML_train.csv', 'data_set_ALL_AML_independent.csv', 'train_scaled.csv', 'test_scaled.csv']


# Setup môi trường

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
import warnings
from sklearn.decomposition import PCA
import joblib
warnings.filterwarnings('ignore')

# Tải dữ liệu

In [7]:
train_path = "/content/drive/MyDrive/Project Machine Learning/data_set_ALL_AML_train.csv"
test_path  = "/content/drive/MyDrive/Project Machine Learning/data_set_ALL_AML_independent.csv"
actual_path = "/content/drive/MyDrive/Project Machine Learning/actual.csv"

# Load dữ liệu
df_train = pd.read_csv(train_path, index_col=0)  # index_col=0 nếu cột đầu là Gene
df_test  = pd.read_csv(test_path, index_col=0)
df_actual = pd.read_csv(actual_path)  # actual.csv nếu cần dùng labels

# Tiền Xử Lý Dữ Liệu (Clean + Transpose + Z-score + PCA)

In [8]:
def clean_gene_expression(df):
    drop_cols = [
        col for col in df.columns
        if "Gene Description" in col
        or "Gene Accession Number" in col
        or "call" in col.lower()
    ]
    df_clean = df.drop(columns=drop_cols, errors='ignore')
    return df_clean

# Clean
df_train_clean = clean_gene_expression(df_train)
df_test_clean  = clean_gene_expression(df_test)

# Transpose
X_train = df_train_clean.T
X_test  = df_test_clean.T

print("Train shape after transpose:", X_train.shape)
print("Test shape after transpose:", X_test.shape)

# Z-score normalization
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

print("Scaled train shape:", X_train_scaled.shape)
print("Scaled test shape:", X_test_scaled.shape)

# PCA
pca = PCA(n_components=0.95, random_state=42)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca  = pca.transform(X_test_scaled)

print("Train PCA shape:", X_train_pca.shape)
print("Test PCA shape:", X_test_pca.shape)
print("Number of PCs kept:", pca.n_components_)

# Đưa PCA về DataFrame
pc_cols = [f"PC{i+1}" for i in range(X_train_pca.shape[1])]

X_train_pca_df = pd.DataFrame(
    X_train_pca,
    index=X_train.index,
    columns=pc_cols
)

X_test_pca_df = pd.DataFrame(
    X_test_pca,
    index=X_test.index,
    columns=pc_cols
)

# LƯU FILE = SAU PCA
X_train_out = X_train_pca_df.copy()
X_train_out.insert(0, "Sample_ID", X_train_out.index)
X_train_out.to_csv(
    "/content/drive/MyDrive/Project Machine Learning/train_scaled.csv",
    index=False
)

X_test_out = X_test_pca_df.copy()
X_test_out.insert(0, "Sample_ID", X_test_out.index)
X_test_out.to_csv(
    "/content/drive/MyDrive/Project Machine Learning/test_scaled.csv",
    index=False
)

Train shape after transpose: (38, 7129)
Test shape after transpose: (34, 7129)
Scaled train shape: (38, 7129)
Scaled test shape: (34, 7129)
Train PCA shape: (38, 32)
Test PCA shape: (34, 32)
Number of PCs kept: 32


# BASE MODELS - K-MEANS++

# BASE MODELS - HIERARCHICAL

# BASE MODELS - GMM

# ENSEMBLE CLUSTERING

#  TEST ON INDEPENDENT

#  EVALUATION